Questo notebook integra i dati di ANAC con quelli di IndicePA e contiene una serie di analisi descrittive. 

Output: *data/anac/anac_indicepa.tsv* contiene tutti i bandi di gara avvenuti nel 2017 da parte di PA censite in indicePA.

**Note**
1. Sono stati estratti un totale di 3913440 gare, di cui 1842313 avvenute nell'anno 2017. Si procede all'analisi descrittiva di quest'ultimi bandi.
2. Molti data record contengono valori inconsistenti dovuti a typo (e.g. date con valori errate - 202/02/2016) o ad omissioni da parte delle PA (e.g. codici fiscali di strutture proponenti aventi valori nulli o uguali a *0*, codici fiscali di aggiudicatari offuscati o inesistenti). 


In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
f_anac = "data/anac/anacDataset_postprocessed.tsv"
#df = pd.read_csv(f_anac, sep="\t", error_bad_lines=False, parse_dates=True)
df = pd.read_csv(f_anac, sep="\t", parse_dates=True, dtype= {"cfStrutturaProponente": str})

In [3]:
#ci sono typo in alcune date (e.g. 202/02/2016)
df['dataInizio'] = pd.to_datetime(df['dataInizio'], errors='coerce', format="%d/%m/%Y")
df['dataUltimazione'] = pd.to_datetime(df['dataUltimazione'], errors='coerce', format="%d/%m/%Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3913440 entries, 0 to 3913439
Data columns (total 17 columns):
cig                                  object
cfStrutturaProponente                object
denominazioneStrutturaProponente     object
oggetto                              object
sceltaContraente                     object
importoAggiudicazione                float64
importoSommeLiquidate                float64
dataInizio                           datetime64[ns]
dataUltimazione                      datetime64[ns]
jsonPartecipanti                     object
jsonAggiudicatari                    object
totalePartecipanti                   int64
totaleAggiudicatari                  int64
totaleRaggruppamentopartecipanti     int64
totaleRaggruppamentoAggiudicatari    int64
cfPrimoaggiudicatario                object
denominazionePrimoaggiudicatario     object
dtypes: datetime64[ns](2), float64(2), int64(4), object(9)
memory usage: 507.6+ MB


## Analisi gare 2017

In [5]:
is_2017 = (df['dataInizio'] >= '01/01/2017') & (df['dataInizio'] <= '31/12/2017')
df_2017 = df.loc[is_2017]
df_2017.shape

(1948850, 17)

In [6]:
df_cfPrimoaggiudicatario_count = df_2017[["cfStrutturaProponente", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"])["cfPrimoaggiudicatario"].agg(["count"]).reset_index(0)
#df_cfPrimoaggiudicatario_count.head(3)
df_cfPrimoaggiudicatario_sum = df_2017[["importoSommeLiquidate","importoAggiudicazione", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"]).sum().reset_index()
#df_cfPrimoaggiudicatario_sum.head(3)

In [7]:
print("df_cfPrimoaggiudicatario_count", df_cfPrimoaggiudicatario_count.shape)
print("df_cfPrimoaggiudicatario_sum", df_cfPrimoaggiudicatario_sum.shape)

df_cfPrimoaggiudicatario_count (327702, 2)
df_cfPrimoaggiudicatario_sum (327702, 3)


In [8]:
df_denominazione_aggiudicatario = df_2017[["cfPrimoaggiudicatario","denominazionePrimoaggiudicatario"]]
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario.set_index("cfPrimoaggiudicatario")
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]
#df_denominazione_aggiudicatario.head(3)

In [9]:
df_join = df_cfPrimoaggiudicatario_count.set_index('cfPrimoaggiudicatario').join(df_cfPrimoaggiudicatario_sum.set_index('cfPrimoaggiudicatario'))
#df_join.head(3)
df_join.shape

(327702, 3)

In [10]:
#Elimino i duplicati dalle denominazioni
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]
df_denominazione_aggiudicatario.shape

(327703, 1)

In [11]:
final_df = pd.merge(df_denominazione_aggiudicatario, df_join, left_index=True, right_index=True, how='inner')
final_df.shape

(327702, 4)

In [12]:
final_df.head(2)

,denominazionePrimoaggiudicatario,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,,
09487440159,Sicond srl,1,650.0,650.0
09606050962,Kresco Group Multibrand srl,1,20050.0,20050.0


## Top aziende, nel 2017, aggiudicatare in termini di gare vinte e importo aggiudicazione 

**Note**

1. Gli aggiudicatari dei bandi di gara possono essere sia singole aziende che raggruppamenti. In quest'ultimo caso il campo *cfPrimoaggiudicatario* e il campo *denominazionePrimoaggiudicatario* avranno valore *Raggruppamento*;
2. Ci sono 74668 bandi di gara vinti da raggruppamenti. Si dedica a questi data record una successiva analisi.

In [13]:
top_aggiuficatari = final_df.sort_values(['count', "importoAggiudicazione"], ascending=False)
top_aggiuficatari.head()

,denominazionePrimoaggiudicatario,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,,
Raggruppamento,Raggruppamento,80132,1.430562e+09,6.056019e+09
00150470342,Gruppo Spaggiari Parma SpA,15047,7.601570e+06,8.399208e+06
00488410010,TELECOM ITALIA SpA,11842,3.750786e+07,1.265512e+08
02027040019,CDATABORGIONE CENTRO DIDATTICO SRL,7537,3.424255e+06,3.992890e+06
03222970406,MyO Srl,7206,4.166970e+06,8.217086e+06


## Integrazione dati ANAC con IndicePA

Il file *"data/indicePA/indicePA.tsv"* è stato estratto a partire dal notebook *Analisi IndicePA*.

**Note**

- Anno 2017: Su 1.842.313 bandi di gara, solo 1.321.761 hanno codici fiscali di strutture proponenti validi. Questo è dovuto al fatto che alcuni codici fiscali sono errati, mentre altri sono codici fiscali di SPA (non censite in IndicePA). Il dataframe è salvato in *data/anac/anac_indicepa_2017.tsv*
- Tutti gli anni: Su 3.913.440  bandi di gara, 2.682.931 hanno codici fiscali di strutture proponenti validi.

In [24]:
f_indicePA = "data/indicePA/indicePA.tsv"
#df_indicePA = pd.read_csv(f_indicePA, sep="\t")
df_indicePA = pd.read_csv(f_indicePA, sep="\t",dtype={'cf': str})

In [25]:
df_indicePA = df_indicePA.rename(columns={'cf': 'cfStrutturaProponente'})
df_indicePA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941 entries, 0 to 13940
Data columns (total 8 columns):
cfStrutturaProponente    13941 non-null object
cod_amm                  13941 non-null object
regione                  13941 non-null object
provincia                13493 non-null object
comune                   13941 non-null object
indirizzo                13941 non-null object
tipologia_istat          13941 non-null object
tipologia_amm            13941 non-null object
dtypes: object(8)
memory usage: 871.4+ KB


In [27]:
df_indicePA.head(2)

,cfStrutturaProponente,cod_amm,regione,provincia,comune,indirizzo,tipologia_istat,tipologia_amm
0,81002250892,istsc_srpc070006,Sicilia,SR,Augusta,"Via Strazzulla, 10",Istituti di Istruzione Statale di Ogni Ordine ...,Pubbliche Amministrazioni
1,83002610844,istsc_agis021005,Sicilia,AG,Ribera,Piazza Zamenhof 1,Istituti di Istruzione Statale di Ogni Ordine ...,Pubbliche Amministrazioni


### ANNO 2017

In [28]:
print(df_indicePA.shape)
print(df_2017.shape)

(13941, 7)
(1948850, 16)


In [33]:
#df_2017_indicePA = pd.merge(df_2017, df_indicePA, left_index=True, right_index=True, how='inner')
df_2017_indicePA = pd.merge(df_2017, df_indicePA, left_on="cfStrutturaProponente", right_on="cfStrutturaProponente", how='inner')
df_2017_indicePA.shape

(1713154, 24)

In [34]:
diff = df_2017.shape[0] - df_2017_indicePA.shape[0]
print("Gare di PA non recensite su indicePA: ", diff)

Gare di PA non recensite su indicePA:  235696


In [49]:
#df_2017_indicePA2 = pd.merge(df_2017, df_indicePA, left_on="cfStrutturaProponente", right_on="cfStrutturaProponente", how='left')
#df_2017_indicePA2.head()

In [50]:
df_2017_indicePA.head(2)

,cfStrutturaProponente,cig,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,...,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario,cod_amm,regione,provincia,comune,indirizzo,tipologia_istat,tipologia_amm
0,07640210964,0000000000,AMCeM Azienda Multiservizi Cerro Maggiore,Impianto di condizionamento,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,650.0,650.0,2017-01-01,2019-12-31,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,09487440159,Sicond srl,amcma,Lombardia,MI,Cerro Maggiore,"Piazza Aldo Moro,1",Altri Enti Locali,Pubbliche Amministrazioni
1,07640210964,0000000000,AMCeM Azienda Multiservizi Cerro Maggiore,RESTYLING DELLA FARMACIA COMUNALE,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,20050.0,20050.0,2017-01-29,2017-01-29,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,09606050962,Kresco Group Multibrand srl,amcma,Lombardia,MI,Cerro Maggiore,"Piazza Aldo Moro,1",Altri Enti Locali,Pubbliche Amministrazioni


In [51]:
df_2017_indicePA.to_csv("data/anac/anac_indicepa_2017.tsv", sep="\t")

### Tutti gli anni

In [57]:
df_all_indicePA = pd.merge(df, df_indicePA, left_on="cfStrutturaProponente", right_on="cfStrutturaProponente", how='inner')
diff = df.shape[0] - df_all_indicePA.shape[0]
print("Gare di PA non recensite su indicePA: %d (su %d gare totali)" % (diff, df.shape[0]))

Gare di PA non recensite su indicePA: 649738 (su 3913440 gare totali)


In [58]:
df_all_indicePA.to_csv("data/anac/anac_indicepa_all.tsv", sep="\t")